In [ ]:
#필요한 모듈
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import gc
import os
import random
print(os.getcwd())
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)
pd.set_option('mode.chained_assignment',  None)

# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer
# from xgboost import XGBClassifier
# from autogluon.tabular import TabularPredictor
# from sklearn.neighbors import LocalOutlierFactor
# from sklearn.ensemble import IsolationForest
# from sklearn.svm import OneClassSVM
# from autogluon.tabular import TabularDataset, TabularPredictor

# 한글 폰트 사용을 위해서 세팅
# from matplotlib import font_manager, rc, rcParams
# font_path = "C:/Windows/Fonts/malgun.ttf"
# font = font_manager.FontProperties(fname = font_path).get_name()
# rc('font', family = font)

#마이너스 깨짐 현상 해결
# rcParams['axes.unicode_minus'] = False

# Seed 고정
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings(action = 'ignore')

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 위,경도를 합쳐 geometry 열 생성 함수

In [ ]:
def create_geometry_column(input_file, output_file, latitude_column, longitude_column):
    # CSV 파일 읽기
    data = pd.read_csv(input_file)

    # 'geometry' 열 생성
    data['geometry'] = data.apply(lambda x: f"POINT ({x[longitude_column]} {x[latitude_column]})", axis=1)

    # 변경된 데이터를 새로운 CSV 파일로 저장
    data.to_csv(output_file, index=False)

# 함수 사용 예시
# create_geometry_column('원본_파일명.csv', '새로운_파일명.csv', '위도 열 이름', '경도 열 이름')

## 기존 국가지점 번호 인구데이터 불러오기

In [ ]:
# Shapefile 불러오기
shapefile_path = '/content/drive/MyDrive/Machine_Learnig_Project/nlsp_020001001.shp'
pop = gpd.read_file(shapefile_path, encoding = 'UTF-8')

In [ ]:
pop.head()

,gid,lbl,val,geometry
0,다바8516,16173.00,16173.0,"POLYGON ((985000.000 1816000.000, 985000.000 1..."
1,라바0120,31.00,31.0,"POLYGON ((1001000.000 1820000.000, 1001000.000..."
2,다바9614,14729.00,14729.0,"POLYGON ((996000.000 1814000.000, 996000.000 1..."
3,다바8728,N/A,0.0,"POLYGON ((987000.000 1828000.000, 987000.000 1..."
4,다바9716,79.00,79.0,"POLYGON ((997000.000 1816000.000, 997000.000 1..."


In [ ]:
pop.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 632 entries, 0 to 631
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   gid       632 non-null    object  
 1   lbl       474 non-null    object  
 2   val       474 non-null    float64 
 3   geometry  632 non-null    geometry
dtypes: float64(1), geometry(1), object(2)
memory usage: 19.9+ KB


## 추가 데이터들 불러오기

In [ ]:
hsm = pd.read_csv('/content/drive/MyDrive/Machine_Learnig_Project/대전_의원.csv')
hsm.head()

,의료기관명,의료기관주소,시군구,읍면동,geometry,개수_x
0,더바른의원,"대전광역시 유성구 테크노중앙로 47, 301호, 304호 (관평동)",유성구,관평동,POINT (127.3913122 36.4270386),53
1,동행어린이치과의원,"대전광역시 유성구 테크노중앙로 38, 4층 (관평동)",유성구,관평동,POINT (127.3896988 36.4267007),53
2,더오래치과의원,"대전광역시 유성구 테크노4로 115, 402호 (관평동)",유성구,관평동,POINT (127.3891566 36.4243967),53
3,엠치과의원,"대전광역시 유성구 관평2로 32, 4층 (관평동)",유성구,관평동,POINT (127.3893377 36.4239688),53
4,애플치과의원,"대전광역시 유성구 테크노4로 105, T PLAZA 4층 410,411호 (관평동)",유성구,관평동,POINT (127.3896414 36.4251117),53


In [ ]:
hbm = pd.read_csv('/content/drive/MyDrive/Machine_Learnig_Project/대전_병원.csv')
hbm.head()

,의료기관명,의료기관주소,시군구,읍면동,geometry,개수
0,대전세종충남·넥슨후원 공공어린이재활병원,"대전광역시 서구 도안중로 133-0 (관저동,대전세종충남넥슨후원공공어린이재활병원)",서구,관저동,POINT (127.3453185 36.3126418),1
1,밝은누리안과병원,대전광역시 서구 둔산로 61 (둔산동),서구,둔산동,POINT (127.3807575 36.35143619999999),9
2,마라톤정형외과병원,"대전광역시 서구 둔산남로 87, 2~6층 (둔산동)",서구,둔산동,POINT (127.387417 36.3493109),9
3,글로벌삼육오엠씨병원,"대전광역시 서구 대덕대로 194, 백마빌딩 7~10층 (둔산동)",서구,둔산동,POINT (127.3782111 36.3501571),9
4,킴벨피부과병원,"대전광역시 서구 대덕대로 238, 2층 (둔산동)",서구,둔산동,POINT (127.379977 36.3541137),9


In [ ]:
hsm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2182 entries, 0 to 2181
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   의료기관명     2182 non-null   object
 1   의료기관주소    2182 non-null   object
 2   시군구       2182 non-null   object
 3   읍면동       2182 non-null   object
 4   geometry  2182 non-null   object
 5   개수_x      2182 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 102.4+ KB


In [ ]:
hbm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   의료기관명     87 non-null     object
 1   의료기관주소    87 non-null     object
 2   시군구       87 non-null     object
 3   읍면동       87 non-null     object
 4   geometry  87 non-null     object
 5   개수        87 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 4.2+ KB


### 좌표체계 맞추기(의원_수)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [ ]:
# 좌표 정보를 Point 객체로 변환
geometry = hsm['geometry'].apply(lambda x: Point(map(float, x[7:-1].split())))

# 새로운 GeoDataFrame 생성
ghsm = gpd.GeoDataFrame(hsm, geometry=geometry)

In [ ]:
ghsm.geometry = ghsm.geometry.set_crs("EPSG:4326")

In [ ]:
ghsm.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2182 entries, 0 to 2181
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   의료기관명     2182 non-null   object  
 1   의료기관주소    2182 non-null   object  
 2   시군구       2182 non-null   object  
 3   읍면동       2182 non-null   object  
 4   geometry  2182 non-null   geometry
 5   개수_x      2182 non-null   int64   
dtypes: geometry(1), int64(1), object(4)
memory usage: 102.4+ KB


### 인구 수 데이터 좌표체계 맞추기

In [ ]:
pop.geometry = pop.geometry.to_crs("EPSG:4326")

In [ ]:
pop.head()

,gid,lbl,val,geometry
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350..."
1,라바0120,31.00,31.0,"POLYGON ((127.51115 36.37745, 127.51115 36.386..."
2,다바9614,14729.00,14729.0,"POLYGON ((127.45544 36.32335, 127.45543 36.332..."
3,다바8728,N/A,0.0,"POLYGON ((127.35493 36.44949, 127.35492 36.458..."
4,다바9716,79.00,79.0,"POLYGON ((127.46657 36.34139, 127.46657 36.350..."


In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon

df = gpd.sjoin(pop, ghsm, how='left', op="contains")
df.dropna(inplace=True)

In [ ]:
df

,gid,lbl,val,geometry,index_right,의료기관명,의료기관주소,시군구,읍면동,개수_x
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350...",62.0,하얀이치과의원,"대전광역시 유성구 도안대로 511-13, 3층 (상대동)",유성구,상대동,13.0
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350...",59.0,도안한의원,"대전광역시 유성구 도안대로 511-13, 202호 (상대동)",유성구,상대동,13.0
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350...",64.0,에이케이플란트치과의원,"대전광역시 유성구 도안대로 511-7, 4층 (상대동)",유성구,상대동,13.0
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350...",56.0,고려정형외과의원,"대전광역시 유성구 도안대로 511-7, 춘추빌딩동 3층 (상대동)",유성구,상대동,13.0
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350...",63.0,지성연합치과의원,"대전광역시 유성구 도안대로 511-6 (상대동,3층)",유성구,상대동,13.0
...,...,...,...,...,...,...,...,...,...,...
471,다바9017,33315.00,33315.0,"POLYGON ((127.38855 36.35036, 127.38854 36.359...",1690.0,최.박치과의원,"대전광역시 서구 문예로 69, 7층 (둔산동, 오성빌딩)",서구,둔산동,306.0
471,다바9017,33315.00,33315.0,"POLYGON ((127.38855 36.35036, 127.38854 36.359...",1691.0,김병호내과의원,"대전광역시 서구 문예로 69, 5층 (둔산동)",서구,둔산동,306.0
471,다바9017,33315.00,33315.0,"POLYGON ((127.38855 36.35036, 127.38854 36.359...",1712.0,박정기치과의원,대전광역시 서구 문예로 73 (둔산동),서구,둔산동,306.0
471,다바9017,33315.00,33315.0,"POLYGON ((127.38855 36.35036, 127.38854 36.359...",1741.0,조석원치과의원,"대전광역시 서구 청사로 254, 201호 (둔산동, 둥지아파트)",서구,둔산동,306.0


In [ ]:
# CSV 파일을 Google 드라이브에 저장
file_path = '/content/drive/My Drive/Machine_Learnig_Project/sh_data.csv'
df.to_csv(file_path, index=False)

## grid 별로 그룹화 하고 갯수를 세서 train_data에 병합(merge)

### 좌표체계 맞추기(병원_수)

In [ ]:
# 좌표 정보를 Point 객체로 변환
geometry = hbm['geometry'].apply(lambda x: Point(map(float, x[7:-1].split())))

# 새로운 GeoDataFrame 생성
ghbm = gpd.GeoDataFrame(hbm, geometry=geometry)

In [ ]:
ghbm.geometry = ghbm.geometry.set_crs("EPSG:4326")

In [ ]:
ghbm.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   의료기관명     87 non-null     object  
 1   의료기관주소    87 non-null     object  
 2   시군구       87 non-null     object  
 3   읍면동       87 non-null     object  
 4   geometry  87 non-null     geometry
 5   개수        87 non-null     int64   
dtypes: geometry(1), int64(1), object(4)
memory usage: 4.2+ KB


### 인구 수 데이터 좌표체계 맞추기

In [ ]:
pop.head()

,gid,lbl,val,geometry
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350..."
1,라바0120,31.00,31.0,"POLYGON ((127.51115 36.37745, 127.51115 36.386..."
2,다바9614,14729.00,14729.0,"POLYGON ((127.45544 36.32335, 127.45543 36.332..."
3,다바8728,N/A,0.0,"POLYGON ((127.35493 36.44949, 127.35492 36.458..."
4,다바9716,79.00,79.0,"POLYGON ((127.46657 36.34139, 127.46657 36.350..."


In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon

df1 = gpd.sjoin(pop, ghbm, how='left', op="contains")
df1.dropna(inplace=True)

In [ ]:
df1

,gid,lbl,val,geometry,index_right,의료기관명,의료기관주소,시군구,읍면동,개수
27,다바9503,249.00,249.0,"POLYGON ((127.44437 36.22418, 127.44436 36.233...",64.0,보광노인전문병원,대전광역시 동구 산내로560번길 18-11 (상소동),동구,상소동,1.0
33,다바9428,3804.00,3804.0,"POLYGON ((127.43305 36.44956, 127.43304 36.458...",40.0,이엘치과병원,"대전광역시 대덕구 신탄진로 782 (신탄진동, 1,2,3,4,5층)",대덕구,신탄진동,2.0
36,다바9116,6374.00,6374.0,"POLYGON ((127.39971 36.34135, 127.39970 36.350...",36.0,평화요양병원,대전광역시 중구 대전천서로 745 (중촌동),중구,중촌동,2.0
50,다바9612,7385.00,7385.0,"POLYGON ((127.45545 36.30532, 127.45544 36.314...",51.0,새봄요양병원,대전광역시 동구 신기로 123-20 (가오동),동구,가오동,1.0
51,다바9613,15255.00,15255.0,"POLYGON ((127.45544 36.31434, 127.45544 36.323...",54.0,대전그린의료소비자생활협동조합 그린요양병원,대전광역시 동구 옥천로176번길 15-4 (판암동),동구,판암동,1.0
...,...,...,...,...,...,...,...,...,...,...
414,다바9319,17013.00,17013.0,"POLYGON ((127.42197 36.36841, 127.42196 36.377...",43.0,근로복지공단대전병원,대전광역시 대덕구 계족로 637 (법동),대덕구,법동,1.0
419,다바9517,18974.00,18974.0,"POLYGON ((127.44428 36.35040, 127.44427 36.359...",39.0,홍제요양병원,대전광역시 대덕구 동서대로 1805 (송촌동),대덕구,송촌동,3.0
445,다바9714,487.00,487.0,"POLYGON ((127.46658 36.32336, 127.46657 36.332...",53.0,대전동부요양병원,대전광역시 동구 동부로 150 (용운동),동구,용운동,1.0
448,다바9218,11095.00,11095.0,"POLYGON ((127.41083 36.35939, 127.41082 36.368...",46.0,의료법인 의명의료재단 대전병원,대전광역시 대덕구 대전로 1215 (오정동),대덕구,오정동,1.0


In [ ]:
# CSV 파일을 Google 드라이브에 저장
file_path = '/content/drive/My Drive/Machine_Learnig_Project/bh_data.csv'
df1.to_csv(file_path, index=False)

## 버스 정류장 데이터 합치기

In [ ]:
bus = pd.read_csv('/content/drive/MyDrive/Machine_Learnig_Project/대전광역시_버스정류장 현황_20221215_v1.csv')
bus.head()

,지형지물부호,관리번호,행정읍면동,도엽번호,도로구간번호,공사번호,정류장종류,정류장명,정류장유형,대장초기화여부,위도,경도
0,정류장,2209000001,대전광역시 동구,36710089,14220,STT0001000,버스공용,면허시험장입구,표지설치,1,36.290530,127.463263
1,정류장,2209000002,대전광역시 동구,36710079,20093,STT0002000,시내버스,은어송마을5단지,유개형(도시형),1,36.308424,127.458762
2,정류장,2209000003,대전광역시 동구,36710079,20093,STT0003000,시내버스,은어송마을5단지,유개형(도시형),1,36.308601,127.459336
3,정류장,2209000004,대전광역시 동구,36710089,18474,STT0004000,버스공용,삼익아파트고산사,유개형(도시형),1,36.296939,127.460022
4,정류장,2209000012,대전광역시 동구,36710078,24095,STT0012000,시내버스,신흥동시외버스정류소,유개형(도시형),1,36.322339,127.444750


In [ ]:
# 'geometry' 열 생성
geometry = [Point(xy) for xy in zip(bus['경도'], bus['위도'])]

# GeoDataFrame 생성
crs = {'init': 'epsg:4326'}  # 위경도 좌표계 EPSG 코드
gdf_bus = gpd.GeoDataFrame(bus, crs=crs, geometry=geometry)

# 'geometry' 열의 타입을 geometry로 변경
gdf_bus = gdf_bus.set_geometry('geometry')

In [ ]:
gdf_bus.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1405 entries, 0 to 1404
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   지형지물부호    1405 non-null   object  
 1   관리번호      1405 non-null   int64   
 2   행정읍면동     1405 non-null   object  
 3   도엽번호      1405 non-null   int64   
 4   도로구간번호    1405 non-null   int64   
 5   공사번호      1405 non-null   object  
 6   정류장종류     1405 non-null   object  
 7   정류장명      1405 non-null   object  
 8   정류장유형     1405 non-null   object  
 9   대장초기화여부   1405 non-null   int64   
 10  위도        1405 non-null   float64 
 11  경도        1405 non-null   float64 
 12  geometry  1405 non-null   geometry
dtypes: float64(2), geometry(1), int64(4), object(6)
memory usage: 142.8+ KB


In [ ]:
gdf_bus.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1405 entries, 0 to 1404
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   지형지물부호    1405 non-null   object  
 1   관리번호      1405 non-null   int64   
 2   행정읍면동     1405 non-null   object  
 3   도엽번호      1405 non-null   int64   
 4   도로구간번호    1405 non-null   int64   
 5   공사번호      1405 non-null   object  
 6   정류장종류     1405 non-null   object  
 7   정류장명      1405 non-null   object  
 8   정류장유형     1405 non-null   object  
 9   대장초기화여부   1405 non-null   int64   
 10  위도        1405 non-null   float64 
 11  경도        1405 non-null   float64 
 12  geometry  1405 non-null   geometry
dtypes: float64(2), geometry(1), int64(4), object(6)
memory usage: 142.8+ KB


### 인구 수 데이터 좌표체계 맞추기

In [ ]:
pop.head()

,gid,lbl,val,geometry
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350..."
1,라바0120,31.00,31.0,"POLYGON ((127.51115 36.37745, 127.51115 36.386..."
2,다바9614,14729.00,14729.0,"POLYGON ((127.45544 36.32335, 127.45543 36.332..."
3,다바8728,N/A,0.0,"POLYGON ((127.35493 36.44949, 127.35492 36.458..."
4,다바9716,79.00,79.0,"POLYGON ((127.46657 36.34139, 127.46657 36.350..."


In [ ]:
df2 = gpd.sjoin(pop, gdf_bus, how='left', op="contains")
df2.dropna(inplace=True)

In [ ]:
df2

,gid,lbl,val,geometry,index_right,지형지물부호,관리번호,행정읍면동,도엽번호,도로구간번호,공사번호,정류장종류,정류장명,정류장유형,대장초기화여부,위도,경도
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350...",1124.0,정류장,2.209001e+09,대전광역시 유성구,36710064.0,31544.0,STT1299000,마을버스,도안 한라비발디3단지,유개형(도시형),1.0,36.342867,127.333976
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350...",1123.0,정류장,2.209001e+09,대전광역시 유성구,36710064.0,31544.0,STT1298000,마을버스,한라비발디아파트,유개형(도시형),1.0,36.343086,127.333513
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350...",1122.0,정류장,2.209001e+09,대전광역시 유성구,36710064.0,31838.0,STT1297000,마을버스,유성구마을버스,유개형(도시형),1.0,36.345661,127.335504
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350...",937.0,정류장,2.209001e+09,대전광역시 유성구,36710064.0,31563.0,STT1081000,시내버스,트리플시티9단지,유개형(도시형),1.0,36.341452,127.338029
0,다바8516,16173.00,16173.0,"POLYGON ((127.33285 36.34128, 127.33283 36.350...",1151.0,정류장,2.209001e+09,대전광역시 유성구,36710064.0,31563.0,STT1331000,버스공용,트리풀시티9단지,유개형(도시형),1.0,36.341497,127.338713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,다바9017,33315.00,33315.0,"POLYGON ((127.38855 36.35036, 127.38854 36.359...",670.0,정류장,1.015650e+05,대전광역시 서구,36710056.0,36906.0,STT0761000,시내버스,수정타운아파트정문,유개형(도시형),1.0,36.358409,127.395388
473,다바8722,55.00,55.0,"POLYGON ((127.35503 36.39540, 127.35502 36.404...",554.0,정류장,2.209001e+09,대전광역시 유성구,36710045.0,45783.0,STT0626000,버스공용,한국기계연구소,유개형(도시형),1.0,36.395843,127.355845
473,다바8722,55.00,55.0,"POLYGON ((127.35503 36.39540, 127.35502 36.404...",438.0,정류장,1.014340e+05,대전광역시 유성구,36710045.0,17223.0,STT0502000,시내버스,충열사,유개형(도시형),1.0,36.397542,127.356025
473,다바8722,55.00,55.0,"POLYGON ((127.35503 36.39540, 127.35502 36.404...",435.0,정류장,1.014300e+05,대전광역시 유성구,36710035.0,17126.0,STT0499000,시내버스,대전광역시인재개발원,유개형(도시형),1.0,36.401726,127.363241


In [ ]:
# CSV 파일을 Google 드라이브에 저장
file_path = '/content/drive/My Drive/Machine_Learnig_Project/bus_data.csv'
df2.to_csv(file_path, index=False)